In [1]:
from __future__ import print_function 
import os 
from textwrap import wrap
import locale
import pandas as pd 
import numpy as np 

%matplotlib inline 
from matplotlib import pyplot as plt 

from pandas.plotting import autocorrelation_plot

from statsmodels.tsa import stattools

locale.setlocale(locale.LC_ALL, '')


'LC_CTYPE=en_US.UTF-8;LC_NUMERIC=it_IT.UTF-8;LC_TIME=it_IT.UTF-8;LC_COLLATE=en_US.UTF-8;LC_MONETARY=it_IT.UTF-8;LC_MESSAGES=en_US.UTF-8;LC_PAPER=it_IT.UTF-8;LC_NAME=it_IT.UTF-8;LC_ADDRESS=it_IT.UTF-8;LC_TELEPHONE=it_IT.UTF-8;LC_MEASUREMENT=it_IT.UTF-8;LC_IDENTIFICATION=it_IT.UTF-8'

In [2]:
# 
# Some common functions.
# 
def remove_tick_lines(which, ax):
    if which == 'y':
        for line in ax.yaxis.get_majorticklines():
            line.set_visible(False)
    elif which == 'x':
        for line in ax.xaxis.get_majorticklines():
            line.set_visible(False)
    else:
        assert False, "Wrong axis parameter."

def every_nth_tick(ax, every_nth = 2):
    for n, label in enumerate(ax.xaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)

def autolabel(rects, ax, dec_no=0):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width() / 2., height + (height * 0.01),
                '%s' % round(height, dec_no),
                ha='center', va='bottom')

def set_axes_common_properties(axe, no_grid=False):
    rv = 1
    try:
        axe.spines['top'].set_visible(False)
        axe.spines['left'].set_visible(False)
        axe.spines['right'].set_visible(False)
        axe.spines['bottom'].set_visible(False)
        if no_grid == False:
            axe.grid(color='#636262', linestyle='-.', linewidth=0.2)
        rv = 0
    except Exception as ex:
        print("Errore - {e}".format(e=str(ex)))
        
    return rv


In [3]:
data_file_path = "/home/giovanni/code-sky/JMeter/MvtGateway/log"
data_file_name = "MvtC2CGateway-C2C-only_20180516_153201-.jtl"

data_file = os.path.join(data_file_path, data_file_name)

In [4]:
dtype_dict = {
}

parse_dates = ["timeStamp"]

columns = ["timeStamp_", "elapsed", "responseCode", "responseMessage","grpThreads","allThreads"]

df = pd.read_csv(data_file, sep='#', lineterminator='\n'
                ,low_memory=False
                ,dtype=dtype_dict)
df["timeStamp_"] = pd.to_datetime(df["timeStamp"], unit='ms')
df.sort_values(by=["timeStamp_"], inplace=True)

# Get just the columns we need.
df = df[columns]

#(df_red.shape, df_red.dtypes)
df.head(2)

,timeStamp_,elapsed,responseCode,responseMessage,grpThreads,allThreads
0,2018-05-16 15:32:03.967,125,200,OK,1,1
1,2018-05-16 15:32:04.100,22,200,OK,1,1


In [18]:
global_statistics = {
     "duration": None
    ,"max threads": None
}
dec_num = 2
ok_codes [200, 202]

In [29]:
# Calculate the test duration in minutes.
begin_d = df["timeStamp_"].min()
end_d = df["timeStamp_"].max()
duration = end_d - begin_d

global_statistics["duration"] = round(duration.total_seconds() / 60, dec_num)

# Max number of threads.
global_statistics["max threads"] = df["allThreads"].max()

# Sample len.
global_statistics["sample num"] = df.shape[0]

# Service time statistics.
global_statistics["max elapsed"] = df["elapsed"].max()
global_statistics["min elapsed"] = df["elapsed"].min()
global_statistics["mean elapsed"] = round(df["elapsed"].mean(), dec_num)
global_statistics["std error elapsed"] = round(df["elapsed"].sem(), dec_num)
global_statistics["quantiles elapsed"] = df["elapsed"].quantile(np.arange(0, 1, 0.1))

# Total transactions (sample not in error per second).
global_statistics["quantiles elapsed"]
global_statistics

{'duration': 15.0,
 'max threads': 150,
 'max elapsed': 3359,
 'min elapsed': 12,
 'mean elapsed': 843.23,
 'std error elapsed': 2.15,
 'sample num': 106622,
 'quantiles elapsed': 0.0      12.0
 0.1      77.0
 0.2     203.0
 0.3     387.0
 0.4     553.0
 0.5     683.0
 0.6     803.0
 0.7    1016.0
 0.8    1366.0
 0.9    2123.0
 Name: elapsed, dtype: float64}

In [31]:
df['responseCode'].value_counts().loc[200]


106622